In [ ]:
# Для конструктора HTML
from jinja2 import Template

In [ ]:
import json

In [ ]:
# Библиотека по работе с API Aspire
import api_aspire as asp

In [ ]:
# Библиотека для конвертации html в pdf
from weasyprint import HTML

In [ ]:
# Конвертирование из html в pdf
def html_to_pdf(html, pdf_path):
    HTML(string=html).write_pdf(pdf_path)
# Конвертирование из url в pdf
def url_to_pdf(url, pdf_path):
    HTML(url).write_pdf(pdf_path)

In [ ]:
# Ответ из API по методу "Категории"
response_infoblocks = asp.requests_aspire('list_block')
# Данные по всем элементам в формате json
response_json_infoblocks = response_infoblocks.json()
infoblocks = dict()
for i in response_json_infoblocks:
    infoblocks[i] = response_json_infoblocks.get(i).get('name')
infoblocks

In [ ]:
# Ответ из API по методу "Категории"
response_category = asp.requests_aspire('categories', id_block='')
# Данные по всем элементам в формате json
response_json_category = response_category.json()
categories = dict()
for i in response_json_category:
    categories[i] = response_json_category.get(i).get('name')
categories

In [ ]:
# Ответ из API по методу "Все элементы"
response_elements = asp.requests_aspire('list_element', id_block=5)
# Данные по всем элементам в формате json
response_json_elements = response_elements.json()
len(response_json_elements)

In [ ]:
# Ответ из API по методу "Все элементы"
response_elements = asp.requests_aspire('search', id_block=5, id_category=18)
# Данные по всем элементам в формате json
response_json_elements = response_elements.json()
len(response_json_elements)

In [ ]:
# Пример данных из API
test_data_json = response_json_elements[list(response_json_elements.keys())[0]]
test_data_json

In [ ]:
# Функция убивающая переносы в описании оффера
def split_tag(value):
    return (value
            .split('<span style="color: #f5f5f5;">')[0]
            .replace('\r\n', '')
            .replace('\t', '')
            .replace('<p>', '')
            .replace('</p>', '')
            .replace('<br>', '')
            .replace('<br />', '')
            .replace('\n', '')
            .replace('<div>', ''))

In [ ]:
# Соответствие названий ключей из API c понятным названием по методу "Все элементы"
with open('info_keys.json') as fk:
    info_keys = json.load(fk)

In [ ]:
# Шаблон html для последующей подстановки переменных
htmk_block = open('html_block.html', 'r').read()

In [ ]:
# Тестовая генерация html на основе тестового json из APi
tm = Template(htmk_block)
msg = tm.render(json_=test_data_json,
                infoblocks= infoblocks,
                categories= categories,
                bool= bool,
                list= list,
                split_tag=split_tag)
                

In [ ]:
# Тестовая конвертация из HTML в PDF
output_path = '/Users/vyg/Downloads/Карточки Aspire/test_custom_html.pdf'
html_to_pdf(msg, output_path)

In [ ]:
# Берем поочередно все офферы и записываем в PDF, каждая категория в отдельный файл
for categ in list(categories.values()):
#for categ in ['Рестораны']:
    msg = '' # HTML блок
    part = 0 # Счетчик итераций записи файла
    ch = 0 # Счетчик подходящих оферов для отщелкивания итерации
    # Берем каждый элемент (оффер)
    for el_id in response_json_elements:
        json = response_json_elements[el_id]
        name_offer = json.get('name')
        category_offer = categories.get(json.get('category'))
        # Сверяем относится ли оффер к итерируемой категории
        if category_offer == categ:
            tm = Template(htmk_block)
            # Собираем html с переменными из json оффера (Элемента)
            msg += tm.render(json_=json,
                            infoblocks= infoblocks,
                            categories= categories,
                            bool= bool,
                            list= list,
                            split_tag= split_tag)
            block_offer = infoblocks.get(json.get('iblock'))
            # Считаем сколько отработали офферов
            ch += 1
            # Записываем 500 офферов в файл, чтобы не было исключения от переполнения
            if ch > 500:
                html_to_pdf(msg, f'/Users/vyg/Downloads/Карточки Aspire/{block_offer}/{categ} {part}.pdf')
                msg = ''
                part += 1
                ch = 0
    # Записываем оставшиеся офферы
    html_to_pdf(msg, f'/Users/vyg/Downloads/Карточки Aspire/{block_offer}/{categ} {part}.pdf')